<a href="https://colab.research.google.com/github/arjoyita/Assignment1/blob/main/final_uip_code_snip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)         # click the link → choose your Google account


Mounted at /content/drive


In [ ]:
!pip -q install pdfplumber python-docx openpyxl tabulate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 100.6 MB/s eta 0:00:00


In [ ]:
import os, re, io, pathlib, tempfile, itertools
import pandas as pd
import pdfplumber
from docx import Document
from tabulate import tabulate



# ---------- the workhorse ----------
def search_folder(folder_path:str, keywords:list[str]):
    rx = {k: re.compile(re.escape(k), re.I) for k in keywords}
    hits = []

    for path in pathlib.Path(folder_path).rglob('*'):
        if not path.is_file():
            continue
        name = path.name.lower()

        try:
            # ---- PDF ----
            if name.endswith('.pdf'):
                with pdfplumber.open(str(path)) as pdf:
                    for pg_idx, page in enumerate(pdf.pages, 1):
                        text = page.extract_text() or ""
                        for kw, r in rx.items():
                            if r.search(text):
                                hits.append([kw, str(path.relative_to(folder_path)),
                                             f'page {pg_idx}', scrub(text[:120])])



            # ---- Word (.docx) ----
            elif name.endswith('.docx'):
                doc = Document(path)
                for p_idx, para in enumerate(doc.paragraphs, 1):
                    text = para.text
                    for kw, r in rx.items():
                        if r.search(text):
                            hits.append([kw, str(path.relative_to(folder_path)),
                                         f'paragraph {p_idx}', scrub(text[:120])])

            # ---- Plain text (.txt) ----
            elif name.endswith('.txt'):
                with open(path, encoding='utf-8', errors='ignore') as fh:
                    for ln, line in enumerate(fh, 1):
                        for kw, r in rx.items():
                            if r.search(line):
                                hits.append([kw, str(path.relative_to(folder_path)),
                                             f'line {ln}', scrub(line.strip()[:120])])

            # (Google Docs / Sheets files are pointers *.gdoc / *.gsheet – cannot be read when mounted;
            #   you’d have to export them first via the Drive API.)
        except Exception as e:
            print('⚠️ Skipped', path.name, '→', e)

    return hits


In [ ]:
# 1. Can Python see the parent folder at all?
import os
print("Exists? ->", os.path.exists("/content/drive/MyDrive/UIP - OPM"))

# 2. Shell listing (immediate feedback)
!ls -la "/content/drive/MyDrive/UIP - OPM"


Exists? -> True
lrw------- 1 root root 0 Jul  8 19:48 '/content/drive/MyDrive/UIP - OPM' -> '/content/drive/.shortcut-targets-by-id/1JjCmeJVsr6sgtXLQrxdPn1p_Q-BStEiU/UIP - OPM'


In [ ]:
import logging, warnings
# 🔧  SET THIS TO YOUR FOLDER  🔧
FOLDER_PATH = "/content/drive/MyDrive/UIP - OPM"
# List the first few files we *think* are in the folder
import pathlib, itertools, os

print("Folder exists?  ", os.path.exists(FOLDER_PATH))
sample = list(itertools.islice(pathlib.Path(FOLDER_PATH).rglob('*.*'), 10))
for p in sample:
    print(" •", p.relative_to(FOLDER_PATH))
print("Total files found:", len(list(pathlib.Path(FOLDER_PATH).rglob('*.*'))))


Folder exists?   True
 • 0-REDACTION of DOCUMENTS for FOIA.docx
 • Acknow-to-Requester-Rev-5.14.docx
 • Training on Requests.gdoc
 • OPM UIPA Requests.xlsx
 • OIP-25-003-JustinWenig_Scholarship-DonorMGT_19May25/FOIA JustinWenig_19May25.xlsx
 • OIP-25-003-JustinWenig_Scholarship-DonorMGT_19May25/UIPEmailtoJWenig_6-23-25.msg
 • OIP-25-001-JustinWenig_LMS-CRM_9Apr25/EmailRequest_Uniform Information Request_ Vendor Contracts.pdf
 • OIP-25-001-JustinWenig_LMS-CRM_9Apr25/Propriety Indication - Completed 04.30.25.xlsx
 • OIP-25-001-JustinWenig_LMS-CRM_9Apr25/NTR J. Wenig 5.7.25.pdf
 • OIP-25-001-JustinWenig_LMS-CRM_9Apr25/Fwd_ Fwd_ Uniform Information Request_ Vendor Contracts.pdf
Total files found: 658


In [ ]:
import pathlib, time
ROOT_PATH = pathlib.Path("/content/drive/MyDrive/UIP - OPM")

t0 = time.time()
subdirs = []
for d in ROOT_PATH.iterdir():
    if d.is_dir():
        subdirs.append(d.name)
        print("  •", d.name)
    # safety: bail after 10 s so we never hang forever
    if time.time() - t0 > 10:
        print("⏳ Listing is taking unusually long; stopping early.")
        break

print(f"\nFound {len(subdirs)} first-level folders in",
      round(time.time() - t0, 2), "seconds.")


  • JustinWenig
  • OIP-25-003-JustinWenig_Scholarship-DonorMGT_19May25
  • OIP-25-001-JustinWenig_LMS-CRM_9Apr25
  • OIP 25-017-Justin Wenig_3Jun25
  • Z-Archived
  • OIP-25-012-Kayla Hitt
  • OIP 25-0114-DocMirino
  • Notice to Requester 

Found 8 first-level folders in 0.01 seconds.


In [ ]:
!sudo apt-get -qq update
!sudo apt-get -qq install poppler-utils tesseract-ocr
!pip -q install pdfplumber pdf2image pytesseract pandas openpyxl



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Privacy Sweep → Per-Match Excel with Exact-Hit Snippets    ║
# ╚══════════════════════════════════════════════════════════════╝
import pathlib, re, pdfplumber, pandas as pd, logging, warnings, pytesseract, tempfile
from pdf2image import convert_from_path
from IPython.display import display

# ── Respect existing settings or define defaults ───────────────
try:
    ROOT_PATH
except NameError:
    ROOT_PATH = pathlib.Path("/content/drive/MyDrive/UIP - OPM")
try:
    SUB_FOLDER_KEY
except NameError:
    SUB_FOLDER_KEY = "OIP"
try:
    SNIP_LEN
except NameError:
    SNIP_LEN = 90

logging.getLogger("pdfminer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="pdfminer")

# ── Regexes ────────────────────────────────────────────────────
RX_EIN_FEIN    = re.compile(r"\b(?:FE?IN)[^\d]{0,6}?(\d{2}[- ]?\d{7})\b|\b(\d{2}[- ]?\d{7})\b", re.I)
RX_PROPRIETARY = re.compile(r"\bProprietary\b", re.I)
RX_EMAIL       = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.I)

# Robust US phone (handles +1, (), hyphens, dots, spaces, ext/x/#, NBSP and common Unicode dashes)
_SEP_CLASS = r"[ \t.\-\u00A0\u2010\u2011\u2012\u2013\u2014\u2015]"
RX_PHONE = re.compile(
    (
        r"(?<!\w)"                        # no letter/number before
        r"(?:\+?1{sep}?)?"                # optional +1 country code
        r"(?:\(?\d{{3}}\)?{sep}?)"        # area code with/without ()
        r"\d{{3}}{sep}?\d{{4}}"           # local number
        r"(?:\s*(?:ext\.?|x|#)\s*\d{{1,5}})?"  # optional extension
        r"(?!\w)"                         # no letter/number after
    ).format(sep=_SEP_CLASS)
)

# Simple US street address w/ state+ZIP (tune as needed)
RX_ADDRESS     = re.compile(
    r"\b\d+\s+[A-Za-z0-9.'\-]+\s+(?:St\.?|Street|Ave\.?|Avenue|Rd\.?|Road|Blvd\.?|Boulevard|Ln\.?|Lane|Dr\.?|Drive)"
    r".{0,60}\b[A-Z]{2}\s*\d{5}(?:-\d{4})?\b", re.I
)

# Vendor reference phrases
RX_VENDOR_REF  = re.compile(
    r"\b(?:vendor\s+reference|customer\s+reference|client\s+reference|past\s+performance|"
    r"performance\s+history|experience\s+(?:record|list)|similar\s+projects?|previous\s+(?:work|projects?|contracts?))\b",
    re.I
)

RX_ATTY_CLIENT = re.compile(r"attorney[-\s]?client", re.I)

CHECKS = {
    "EIN/FEIN"    : RX_EIN_FEIN,
    "Proprietary" : RX_PROPRIETARY,
    "Email"       : RX_EMAIL,
    "Phone"       : RX_PHONE,
    "Address"     : RX_ADDRESS,
    "Vendor_Ref"  : RX_VENDOR_REF,
    "Atty_Client" : RX_ATTY_CLIENT,
}

# ── Helpers ────────────────────────────────────────────────────
def _ocr_page(pdf_path, pageno):
    """OCR fallback for a single page (PNG @300dpi → Tesseract)."""
    with tempfile.TemporaryDirectory() as td:
        imgs = convert_from_path(str(pdf_path), dpi=300, first_page=pageno, last_page=pageno)
        if not imgs:
            return ""
        img_path = pathlib.Path(td) / f"p{pageno}.png"
        imgs[0].save(img_path)
        return pytesseract.image_to_string(str(img_path))

def _make_snippet(text, span, radius=SNIP_LEN//2):
    """Context snippet with the exact hit wrapped in ⟦ ⟧."""
    if not text:
        return ""
    a0, b0 = span
    a = max(0, a0 - radius)
    b = min(len(text), b0 + radius)
    prefix = "…" if a > 0 else ""
    suffix = "…" if b < len(text) else ""
    return f"{prefix}{text[a:a0]}⟦{text[a0:b0]}⟧{text[b0:b]}{suffix}"

# ── Locate target folder & PDFs ────────────────────────────────
root = pathlib.Path(ROOT_PATH)
candidates = [p for p in root.rglob("*") if p.is_dir() and SUB_FOLDER_KEY.lower() in p.name.lower()]
if not candidates:
    raise FileNotFoundError(f"No folder found under {ROOT_PATH} matching: {SUB_FOLDER_KEY}")
target = candidates[0]
pdfs = sorted(target.rglob("*.pdf"))

print(f"Target: {target}")
print(f"PDFs found: {len(pdfs)}")

# ── Scan and collect per-match rows ────────────────────────────
rows = []
for idx, pdf_path in enumerate(pdfs, 1):
    print(f"  [{idx}/{len(pdfs)}] {pdf_path.name}")
    with pdfplumber.open(str(pdf_path)) as pdf:
        for pg_no, page in enumerate(pdf.pages, 1):
            try:
                txt = page.extract_text() or ""
            except Exception:
                txt = ""
            if not txt.strip():
                # OCR fallback
                try:
                    txt = _ocr_page(pdf_path, pg_no)
                except Exception:
                    txt = ""

            # iterate each regex, collect each match as its own row
            for check_name, pattern in CHECKS.items():
                if not isinstance(pattern, re.Pattern):
                    continue
                for m in pattern.finditer(txt):
                    rows.append({
                        "Folder"     : target.name,
                        "File"       : pdf_path.name,
                        "Path"       : str(pdf_path.relative_to(target)),
                        "Page"       : pg_no,
                        "HitType"    : check_name,
                        "MatchedText": m.group(0),
                        "StartIdx"   : m.start(),
                        "EndIdx"     : m.end(),
                        "Snippet"    : _make_snippet(txt, m.span()),
                    })

# ── Output Excel (per-match) ──────────────────────────────────
if rows:
    df = pd.DataFrame(rows)
    display(df.head(50))
    out = target / f"privacy_hits_{target.name}.xlsx"
    df.to_excel(out, index=False)
    print("\n✅ Saved Excel:", out)
else:
    print("\nℹ️  No checklist items flagged in", target)


Target: /content/drive/MyDrive/UIP - OPM/OIP-25-007-MelissaPostle_SODEXO_2Jun25
PDFs found: 33
  [1/33] 2ndCheck_Melissa Postle_8-26-25.pdf
  [2/33] Email Request.pdf
  [3/33] NTR M. Postle 6.30.25.pdf
  [4/33] NTR M. Postle 7.23.25.pdf
  [5/33] NTR M. Postle 8.12.25.pdf
  [6/33] 1.  C160114 - Original Contract.pdf
  [7/33] 2.  C160114 - Mod 1.pdf
  [8/33] 3.  C160114 - Mod 2.pdf
  [9/33] 4.  C160114 - Mod 3.pdf
  [10/33] 5.  C160114 - Mod 4.pdf
  [11/33] 6.  C160114 - Mod 5.pdf
  [12/33] 7.  C160114 - Mod 6.pdf


KeyboardInterrupt: 

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Checkpointed Privacy Sweep  –  resumable per PDF            ║
# ╚══════════════════════════════════════════════════════════════╝
import pathlib, re, csv, pdfplumber, pandas as pd, logging, warnings, pytesseract, tempfile
from pdf2image import convert_from_path
from IPython.display import display

# ── EDIT HERE ──────────────────────────────────────────────────
ROOT_PATH      = pathlib.Path("/content/drive/MyDrive/UIP - OPM")
SUB_FOLDER_KEY = "OIP 25-0114-DocMirino"                 # part/full folder name
SNIP_LEN       = 90                                      # character window around the FIRST priority hit
# ───────────────────────────────────────────────────────────────

logging.getLogger("pdfminer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="pdfminer")

# ── REGEX SETUPS (updated) ─────────────────────────────────────
RX_EIN_FEIN       = re.compile(r"\b(?:FE?IN)[^\d]{0,6}?(\d{2}[- ]?\d{7})\b|\b(\d{2}[- ]?\d{7})\b", re.I)
RX_PROPRIETARY    = re.compile(r"\bProprietary\b", re.I)       # now separate
RX_CONFIDENTIAL   = re.compile(r"\bConfidential\b", re.I)      # now separate
RX_EMAIL          = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.I)

# PHONE: robust (+1, () area code, unicode dashes, separators, optional ext; avoids bare 10-digit runs)
_RX_SEP = r"[ \.\-_/–—]"
RX_PHONE = re.compile(
    r"(?<!\d)"                                 # no digit before
    r"(?:\+?1" + _RX_SEP + r"*)?"              # optional +1
    r"(?:(?:\(\s*\d{3}\s*\)" + _RX_SEP + r"*\d{3}" + _RX_SEP + r"*\d{4})"  # (808) 555-1234
    r"|(?:\d{3}" + _RX_SEP + r"+\d{3}" + _RX_SEP + r"+\d{4}))"              # 808-555-1234 / 808.555.1234 / etc.
    r"(?:"
        + _RX_SEP + r"*(?:x|ext\.?|extension)\s*\d{1,5}"                    # optional extension
    r")?"
    r"(?!\d)", re.I
)

# Heuristic address (kept simple as in your original)
RX_ADDRESS        = re.compile(r"\b\d+\s+\w+\s+(?:St\.?|Street|Ave\.?|Avenue|Rd\.?|Road)\b.*\b[A-Z]{2}\s*\d{5}\b", re.I)

RX_VENDOR_REF     = re.compile(
    r"\b("
    r"vendor\s+reference|references?|reference\s+list|"
    r"customer\s+reference|client\s+reference|"
    r"past\s+performance|performance\s+history|"
    r"experience\s+record|experience\s+list|"
    r"similar\s+projects?|previous\s+(work|projects?|contracts?)|"
    r"completed\s+projects?|worked\s+with|has\s+performed|performed\s+(work|services)"
    r")\b", re.I)

RX_ATTY_CLIENT    = re.compile(r"attorney[-\s]?client", re.I)

PERSONAL_EMAIL_DOMAINS = ("gmail.", "yahoo.", "outlook.", "hotmail.", "icloud.", "protonmail.", "aol.")

# ── OCR helper ─────────────────────────────────────────────────
def ocr_page(pdf_path, num):
    with tempfile.TemporaryDirectory() as td:
        imgs = convert_from_path(str(pdf_path), dpi=300, fmt="png",
                                 first_page=num, last_page=num,
                                 output_folder=td)
        return pytesseract.image_to_string(imgs[0]) if imgs else ""

# ── utilities ──────────────────────────────────────────────────
def _line_col(text:str, idx:int):
    """Return 1-based (line, col) for char index idx."""
    line = text.count("\n", 0, idx) + 1
    last_nl = text.rfind("\n", 0, idx)
    col = (idx - last_nl) if last_nl != -1 else (idx + 1)
    return line, col

def _first_match_span(pat:re.Pattern, text:str):
    m = pat.search(text)
    return (m.start(), m.end(), m.group(0)) if m else None

def _first_personal_email(text:str):
    for m in RX_EMAIL.finditer(text):
        email = m.group(0).lower()
        dom = email.split("@")[-1]
        if email.endswith(".edu"):
            continue
        if any(d in dom for d in PERSONAL_EMAIL_DOMAINS):
            return (m.start(), m.end(), m.group(0))
    return None

def personal_email_present(text:str)->bool:
    return _first_personal_email(text) is not None

def flag_page(txt:str)->dict:
    return {
        "EIN/FEIN"      : bool(RX_EIN_FEIN.search(txt)),
        "Proprietary"   : bool(RX_PROPRIETARY.search(txt)),
        "Confidential"  : bool(RX_CONFIDENTIAL.search(txt)),
        "Personal_Email": personal_email_present(txt),
        "Personal_Phone": bool(RX_PHONE.search(txt)),
        "Personal_Addr" : bool(RX_ADDRESS.search(txt)),
        "Vendor_Ref"    : bool(RX_VENDOR_REF.search(txt)),
        "Atty_Client"   : bool(RX_ATTY_CLIENT.search(txt)),
    }

def locate_hits(txt:str)->dict:
    """
    Return dict of label -> (line, col) and a dict of label -> start index (for snippet centering).
    """
    locs, starts = {}, {}
    # EIN/FEIN
    m = _first_match_span(RX_EIN_FEIN, txt)
    if m:
        line, col = _line_col(txt, m[0]); locs["EIN/FEIN"] = (line, col); starts["EIN/FEIN"] = m[0]
    # Proprietary
    m = _first_match_span(RX_PROPRIETARY, txt)
    if m:
        line, col = _line_col(txt, m[0]); locs["Proprietary"] = (line, col); starts["Proprietary"] = m[0]
    # Confidential
    m = _first_match_span(RX_CONFIDENTIAL, txt)
    if m:
        line, col = _line_col(txt, m[0]); locs["Confidential"] = (line, col); starts["Confidential"] = m[0]
    # Personal Email (first personal)
    m = _first_personal_email(txt)
    if m:
        line, col = _line_col(txt, m[0]); locs["Personal_Email"] = (line, col); starts["Personal_Email"] = m[0]
    # Phone
    m = _first_match_span(RX_PHONE, txt)
    if m:
        line, col = _line_col(txt, m[0]); locs["Personal_Phone"] = (line, col); starts["Personal_Phone"] = m[0]
    # Address
    m = _first_match_span(RX_ADDRESS, txt)
    if m:
        line, col = _line_col(txt, m[0]); locs["Personal_Addr"] = (line, col); starts["Personal_Addr"] = m[0]
    # Vendor references / past performance
    m = _first_match_span(RX_VENDOR_REF, txt)
    if m:
        line, col = _line_col(txt, m[0]); locs["Vendor_Ref"] = (line, col); starts["Vendor_Ref"] = m[0]
    # Attorney-client
    m = _first_match_span(RX_ATTY_CLIENT, txt)
    if m:
        line, col = _line_col(txt, m[0]); locs["Atty_Client"] = (line, col); starts["Atty_Client"] = m[0]
    return locs, starts

def build_location_string(page_no:int, locs:dict)->str:
    # "EIN/FEIN: Page 3, line 12 (char 45); Proprietary: Page 3, line 2 (char 18); Confidential: ..."
    parts = []
    for label, (line, col) in locs.items():
        parts.append(f"{label}: Page {page_no}, line {line} (char {col})")
    return "; ".join(parts)

def build_centered_snippet(txt:str, start_idx:int, window:int)->str:
    if start_idx is None:
        snip = (txt.strip().replace("\n"," ")[:window])
        return snip + ("…" if len(txt) > window else "")
    half = max(12, window // 2)
    a = max(0, start_idx - half)
    b = min(len(txt), start_idx + half)
    snip = txt[a:b].replace("\n", " ").strip()
    prefix = "…" if a > 0 else ""
    suffix = "…" if b < len(txt) else ""
    return prefix + snip + suffix

# ── locate target folder ──────────────────────────────────────
target = next((d for d in ROOT_PATH.iterdir()
               if d.is_dir() and SUB_FOLDER_KEY.lower() in d.name.lower()), None)
if not target:
    raise FileNotFoundError(f"No sub-folder under {ROOT_PATH} contains “{SUB_FOLDER_KEY}”")

print("🔍 Scanning:", target)

done_file   = target / "__done.txt"
running_csv = target / "privacy_hits_running.csv"
final_xlsx  = target / f"privacy_hits_{target.name}.xlsx"

processed = set()
if done_file.exists():
    processed = set(p.strip() for p in done_file.read_text().splitlines() if p.strip())
    print(f"Resume mode: {len(processed)} PDF(s) already done.")

def append_rows_to_csv(path: pathlib.Path, rows, header):
    write_header = not path.exists()
    with open(path, "a", newline='', encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=header)
        if write_header:
            w.writeheader()
        w.writerows(rows)

# ── main scan loop ────────────────────────────────────────────
pdfs = list(target.rglob("*.pdf"))
rows_to_write = []
header_order  = ["File","Page","EIN/FEIN","Proprietary","Confidential","Personal_Email",
                 "Personal_Phone","Personal_Addr","Vendor_Ref","Atty_Client",
                 "Location","Snippet"]

for idx, pdf_path in enumerate(pdfs, 1):
    rel = pdf_path.relative_to(target).as_posix()
    if rel in processed:
        continue   # skip already done

    print(f"[{idx}/{len(pdfs)}] {rel}")
    new_rows = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for pg_no, page in enumerate(pdf.pages, 1):
            txt = page.extract_text() or ""
            if not txt.strip():
                txt = ocr_page(pdf_path, pg_no)

            flags = flag_page(txt)
            if not any(flags.values()):
                continue

            # compute precise locations (line/char) for each category present
            locs, starts = locate_hits(txt)

            # choose FIRST relevant hit for snippet center (Confidential > Proprietary > EIN/FEIN > Vendor_Ref > Personal_Email > Personal_Phone > Personal_Addr > Atty_Client)
            priority = ["Confidential","Proprietary","EIN/FEIN","Vendor_Ref","Personal_Email","Personal_Phone","Personal_Addr","Atty_Client"]
            start_for_snip = None
            for key in priority:
                if flags.get(key) and key in starts:
                    start_for_snip = starts[key]
                    break

            snippet = build_centered_snippet(txt, start_for_snip, SNIP_LEN)
            loc_str = build_location_string(pg_no, {k:v for k,v in locs.items() if flags.get(k)})

            new_rows.append({
                "File"   : rel,
                "Page"   : pg_no,
                **{k: "✔" if v else "" for k,v in flags.items()},
                "Location": loc_str,
                "Snippet": snippet
            })

    # append to CSV and mark done immediately (so restart can continue)
    if new_rows:
        append_rows_to_csv(running_csv, new_rows, header_order)
    with open(done_file, "a") as f:
        f.write(rel + "\n")

# ── convert running CSV → final Excel ─────────────────────────
if running_csv.exists():
    df_final = pd.read_csv(running_csv)
    display(df_final)
    df_final.to_excel(final_xlsx, index=False)
    print("\n✅ Excel saved →", final_xlsx)
else:
    print("\nℹ️  No checklist items flagged in", target)


🔍 Scanning: /content/drive/MyDrive/UIP - OPM/OIP 25-0114-DocMirino
Resume mode: 1 PDF(s) already done.


,File,Page,EIN/FEIN,Proprietary,Personal_Email,Personal_Phone,Personal_Addr,Vendor_Ref,Atty_Client,Snippet
0,2. RFP 22 4815 Salvage and Collection Services...,3,NaN,NaN,NaN,✔,NaN,NaN,NaN,NOTICE TO OFFERORS PROPOSAL FORMS for Request ...
1,2. RFP 22 4815 Salvage and Collection Services...,4,NaN,NaN,NaN,NaN,NaN,✔,NaN,OPRPM FORM 115 (Rev. 09/2005) BUSINESS CLASSIF...
2,2. RFP 22 4815 Salvage and Collection Services...,5,NaN,NaN,NaN,NaN,NaN,✔,NaN,"A. ""SMALL BUSINESS"" SIZE STANDARDS FOR FEDERAL..."
3,2. RFP 22 4815 Salvage and Collection Services...,7,NaN,NaN,NaN,✔,NaN,NaN,NaN,"RFP Issued May 2, 2022 Closing Date for Submis..."
4,2. RFP 22 4815 Salvage and Collection Services...,9,NaN,NaN,NaN,✔,NaN,NaN,NaN,"Susan Mabe, UH Institutional Support Loan Coll..."
5,2. RFP 22 4815 Salvage and Collection Services...,25,NaN,NaN,NaN,NaN,NaN,✔,NaN,SECTION 5 SPECIAL PROVISIONS 5.1 SCOPE The Sal...
6,2. RFP 22 4815 Salvage and Collection Services...,26,NaN,✔,NaN,NaN,NaN,NaN,NaN,5.3 CONFIDENTIAL INFORMATION Offeror shall req...



✅ Excel saved → /content/drive/MyDrive/UIP - OPM/OIP 25-0114-DocMirino/privacy_hits_OIP 25-0114-DocMirino.xlsx


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Checkpointed Privacy Sweep – detailed, one-row-per-hit      ║
# ╚══════════════════════════════════════════════════════════════╝
import pathlib, re, csv, pdfplumber, pandas as pd, logging, warnings, pytesseract, tempfile
from pdf2image import convert_from_path
from IPython.display import display

# ── EDIT HERE ──────────────────────────────────────────────────
ROOT_PATH      = pathlib.Path("/content/drive/MyDrive/UIP - OPM")
SUB_FOLDER_KEY = "OIP 25-0114-DocMirino"                 # part/full folder name
SNIP_LEN       = 90                                      # total snippet width around each hit
# ───────────────────────────────────────────────────────────────

logging.getLogger("pdfminer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="pdfminer")

# ── REGEX (updated) ────────────────────────────────────────────
RX_EIN_FEIN       = re.compile(r"\b(?:FE?IN)[^\d]{0,6}?(\d{2}[- ]?\d{7})\b|\b(\d{2}[- ]?\d{7})\b", re.I)
RX_PROPRIETARY    = re.compile(r"\bProprietary\b", re.I)
RX_CONFIDENTIAL   = re.compile(r"\bConfidential\b", re.I)
RX_EMAIL          = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.I)

# PHONE: robust (+1, () area code, unicode dashes, separators, optional ext; avoids bare 10-digit runs)
_RX_SEP = r"[ \.\-_/–—]"
RX_PHONE = re.compile(
    r"(?<!\d)"                                 # no digit before
    r"(?:\+?1" + _RX_SEP + r"*)?"              # optional +1
    r"(?:(?:\(\s*\d{3}\s*\)" + _RX_SEP + r"*\d{3}" + _RX_SEP + r"*\d{4})"  # (808) 555-1234
    r"|(?:\d{3}" + _RX_SEP + r"+\d{3}" + _RX_SEP + r"+\d{4}))"              # 808-555-1234 / 808.555.1234 / etc.
    r"(?:"
        + _RX_SEP + r"*(?:x|ext\.?|extension)\s*\d{1,5}"                    # optional extension
    r")?"
    r"(?!\d)", re.I
)

# Address (kept simple like your original)
RX_ADDRESS        = re.compile(r"\b\d+\s+\w+\s+(?:St\.?|Street|Ave\.?|Avenue|Rd\.?|Road)\b.*\b[A-Z]{2}\s*\d{5}\b", re.I)

RX_VENDOR_REF     = re.compile(
    r"\b("
    r"vendor\s+reference|references?|reference\s+list|"
    r"customer\s+reference|client\s+reference|"
    r"past\s+performance|performance\s+history|"
    r"experience\s+record|experience\s+list|"
    r"similar\s+projects?|previous\s+(work|projects?|contracts?)|"
    r"completed\s+projects?|worked\s+with|has\s+performed|performed\s+(work|services)"
    r")\b", re.I)

RX_ATTY_CLIENT    = re.compile(r"attorney[-\s]?client", re.I)

# ── OCR helper ─────────────────────────────────────────────────
def ocr_page(pdf_path, num):
    with tempfile.TemporaryDirectory() as td:
        imgs = convert_from_path(str(pdf_path), dpi=300, fmt="png",
                                 first_page=num, last_page=num,
                                 output_folder=td)
        return pytesseract.image_to_string(imgs[0]) if imgs else ""

# ── util: line/col + snippet ──────────────────────────────────
def _line_col(text:str, idx:int):
    line = text.count("\n", 0, idx) + 1
    last_nl = text.rfind("\n", 0, idx)
    col = (idx - last_nl) if last_nl != -1 else (idx + 1)
    return line, col

def _center_snip(text:str, start:int, window:int):
    half = max(12, window // 2)
    a = max(0, start - half); b = min(len(text), start + half)
    snip = text[a:b].replace("\n"," ").strip()
    return ("…" if a > 0 else "") + snip + ("…" if b < len(text) else "")

# ── which patterns to emit as rows (order controls display grouping) ─────
HIT_PATTERNS = [
    ("Email",        RX_EMAIL),
    ("Phone",        RX_PHONE),
    ("EIN/FEIN",     RX_EIN_FEIN),
    ("Proprietary",  RX_PROPRIETARY),
    ("Confidential", RX_CONFIDENTIAL),
    ("Address",      RX_ADDRESS),
    ("Vendor Ref",   RX_VENDOR_REF),
    ("Atty Client",  RX_ATTY_CLIENT),
]

# ── locate target folder ──────────────────────────────────────
target = next((d for d in ROOT_PATH.iterdir()
               if d.is_dir() and SUB_FOLDER_KEY.lower() in d.name.lower()), None)
if not target:
    raise FileNotFoundError(f"No sub-folder under {ROOT_PATH} contains “{SUB_FOLDER_KEY}”")

print("🔍 Scanning:", target)

done_file   = target / "__done.txt"
running_csv = target / "privacy_hits_detailed.csv"                # detailed, one-row-per-hit
final_xlsx  = target / f"privacy_hits_detailed_{target.name}.xlsx"

processed = set()
if done_file.exists():
    processed = set(p.strip() for p in done_file.read_text().splitlines() if p.strip())
    print(f"Resume mode: {len(processed)} PDF(s) already done.")

def append_rows_to_csv(path: pathlib.Path, rows, header):
    write_header = not path.exists()
    with open(path, "a", newline='', encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=header)
        if write_header:
            w.writeheader()
        w.writerows(rows)

# ── main scan loop (row per hit) ──────────────────────────────
pdfs = list(target.rglob("*.pdf"))
header_order = ["Folder","File","Path","Page","HitType","MatchedText",
                "StartIdx","EndIdx","Location","Snippet"]

for idx, pdf_path in enumerate(pdfs, 1):
    rel = pdf_path.relative_to(target).as_posix()
    if rel in processed:
        continue

    print(f"[{idx}/{len(pdfs)}] {rel}")
    rows = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for pg_no, page in enumerate(pdf.pages, 1):
            txt = page.extract_text() or ""
            if not txt.strip():
                txt = ocr_page(pdf_path, pg_no)

            if not txt:
                continue

            # find all hits for each pattern, one row per match
            for hit_type, pat in HIT_PATTERNS:
                for m in pat.finditer(txt):
                    s, e = m.span()
                    line, col = _line_col(txt, s)
                    snippet = _center_snip(txt, s, SNIP_LEN)

                    rows.append({
                        "Folder"     : target.name,
                        "File"       : pdf_path.name,
                        "Path"       : rel,
                        "Page"       : pg_no,
                        "HitType"    : hit_type,
                        "MatchedText": m.group(0),
                        "StartIdx"   : s,
                        "EndIdx"     : e,
                        "Location"   : f"Page {pg_no}, line {line} (char {col})",
                        "Snippet"    : snippet
                    })

    # append and checkpoint
    if rows:
        append_rows_to_csv(running_csv, rows, header_order)
    with open(done_file, "a") as f:
        f.write(rel + "\n")

# ── CSV → Excel (preview) ─────────────────────────────────────
if running_csv.exists():
    df_final = pd.read_csv(running_csv)
    display(df_final.head(50))  # quick preview
    df_final.to_excel(final_xlsx, index=False)
    print("\n✅ Excel saved →", final_xlsx)
else:
    print("\nℹ️  No hits found in", target)


🔍 Scanning: /content/drive/MyDrive/UIP - OPM/OIP 25-0114-DocMirino
Resume mode: 1 PDF(s) already done.
[2/19]  Draft Contracts/Exhibit B - GENERAL PROVISIONS.pdf


KeyboardInterrupt: 

master cell


In [ ]:
 # ╔══════════════════════════════════════════════════════════════╗
# ║  Checkpointed Privacy Sweep – detailed, one-row-per-hit      ║
# ╚══════════════════════════════════════════════════════════════╝
import pathlib, re, csv, pdfplumber, pandas as pd, logging, warnings, pytesseract, tempfile
from pdf2image import convert_from_path
from IPython.display import display

# ── EDIT HERE ──────────────────────────────────────────────────
ROOT_PATH      = pathlib.Path("/content/drive/MyDrive/UIP - OPM")
SUB_FOLDER_KEY = "OIP 25-017-Justin Wenig_3Jun25"      # part/full folder name
SNIP_LEN       = 90                            # centered snippet width
RESCAN_ALL     = True                          # <-- set True to ignore __done.txt and rescan everything
RUN_BASENAME   = "privacy_hits_detailed"       # base name for CSV/XLSX
# ───────────────────────────────────────────────────────────────

logging.getLogger("pdfminer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="pdfminer")

# ── REGEX (updated) ────────────────────────────────────────────
RX_EIN_FEIN       = re.compile(r"\b(?:FE?IN)[^\d]{0,6}?(\d{2}[- ]?\d{7})\b|\b(\d{2}[- ]?\d{7})\b", re.I)
RX_PROPRIETARY    = re.compile(r"\bProprietary\b", re.I)
RX_CONFIDENTIAL   = re.compile(r"\bConfidential\b", re.I)
RX_EMAIL          = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.I)

# PHONE: robust (+1, () area code, unicode dashes, separators, optional ext; avoids bare 10-digit runs)
_RX_SEP = r"[ \.\-_/–—]"
RX_PHONE = re.compile(
    r"(?<!\d)"
    r"(?:\+?1" + _RX_SEP + r"*)?"
    r"(?:(?:\(\s*\d{3}\s*\)" + _RX_SEP + r"*\d{3}" + _RX_SEP + r"*\d{4})"
    r"|(?:\d{3}" + _RX_SEP + r"+\d{3}" + _RX_SEP + r"+\d{4}))"
    r"(?:" + _RX_SEP + r"*(?:x|ext\.?|extension)\s*\d{1,5})?"
    r"(?!\d)", re.I
)

RX_ADDRESS        = re.compile(r"\b\d+\s+\w+\s+(?:St\.?|Street|Ave\.?|Avenue|Rd\.?|Road)\b.*\b[A-Z]{2}\s*\d{5}\b", re.I)
RX_VENDOR_REF     = re.compile(
    r"\b("
    r"vendor\s+reference|references?|reference\s+list|"
    r"customer\s+reference|client\s+reference|"
    r"past\s+performance|performance\s+history|"
    r"experience\s+record|experience\s+list|"
    r"similar\s+projects?|previous\s+(work|projects?|contracts?)|"
    r"completed\s+projects?|worked\s+with|has\s+performed|performed\s+(work|services)"
    r")\b", re.I)
RX_ATTY_CLIENT    = re.compile(r"attorney[-\s]?client", re.I)

# ── OCR helper ─────────────────────────────────────────────────
def ocr_page(pdf_path, num):
    with tempfile.TemporaryDirectory() as td:
        imgs = convert_from_path(str(pdf_path), dpi=300, fmt="png",
                                 first_page=num, last_page=num,
                                 output_folder=td)
        return pytesseract.image_to_string(imgs[0]) if imgs else ""

# ── util: line/col + snippet ──────────────────────────────────
def _line_col(text:str, idx:int):
    line = text.count("\n", 0, idx) + 1
    last_nl = text.rfind("\n", 0, idx)
    col = (idx - last_nl) if last_nl != -1 else (idx + 1)
    return line, col

def _center_snip(text:str, start:int, window:int):
    half = max(12, window // 2)
    a = max(0, start - half); b = min(len(text), start + half)
    snip = text[a:b].replace("\n"," ").strip()
    return ("…" if a > 0 else "") + snip + ("…" if b < len(text) else "")

# ── which patterns to emit as rows (order controls grouping) ──
HIT_PATTERNS = [
    ("Email",        RX_EMAIL),
    ("Phone",        RX_PHONE),
    ("EIN/FEIN",     RX_EIN_FEIN),
    ("Proprietary",  RX_PROPRIETARY),
    ("Confidential", RX_CONFIDENTIAL),
    ("Address",      RX_ADDRESS),
    ("Vendor Ref",   RX_VENDOR_REF),
    ("Atty Client",  RX_ATTY_CLIENT),
]

# ── locate target folder ──────────────────────────────────────
target = next((d for d in ROOT_PATH.iterdir()
               if d.is_dir() and SUB_FOLDER_KEY.lower() in d.name.lower()), None)
if not target:
    raise FileNotFoundError(f"No sub-folder under {ROOT_PATH} contains “{SUB_FOLDER_KEY}”")

print("🔍 Scanning:", target)

done_file   = target / "__done.txt"
running_csv = target / f"{RUN_BASENAME}.csv"
final_xlsx  = target / f"{RUN_BASENAME}_{target.name}.xlsx"

processed = set()
if done_file.exists() and not RESCAN_ALL:
    processed = set(p.strip() for p in done_file.read_text().splitlines() if p.strip())

# inventory + progress info
pdfs = list(target.rglob("*.pdf"))
skipped = sum(1 for p in pdfs if p.relative_to(target).as_posix() in processed)
to_scan = [p for p in pdfs if p.relative_to(target).as_posix() not in processed or RESCAN_ALL]

print(f"📄 PDFs found: {len(pdfs)}  |  Skipped (done): {skipped}  |  Will scan now: {len(to_scan)}")
if RESCAN_ALL:
    print("↻ RESCAN_ALL=True → ignoring __done.txt for this run.")

def append_rows_to_csv(path: pathlib.Path, rows, header):
    write_header = not path.exists()
    with open(path, "a", newline='', encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=header)
        if write_header:
            w.writeheader()
        w.writerows(rows)

# ── main scan loop (row per hit) ──────────────────────────────
header_order = ["Folder","File","Path","Page","HitType","MatchedText",
                "StartIdx","EndIdx","Location","Snippet"]

for idx, pdf_path in enumerate(to_scan, 1):
    rel = pdf_path.relative_to(target).as_posix()
    print(f"[{idx}/{len(to_scan)}] {rel}")
    rows = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for pg_no, page in enumerate(pdf.pages, 1):
            txt = page.extract_text() or ""
            if not txt.strip():
                txt = ocr_page(pdf_path, pg_no)
            if not txt:
                continue

            for hit_type, pat in HIT_PATTERNS:
                for m in pat.finditer(txt):
                    s, e = m.span()
                    line, col = _line_col(txt, s)
                    snippet = _center_snip(txt, s, SNIP_LEN)
                    rows.append({
                        "Folder"     : target.name,
                        "File"       : pdf_path.name,
                        "Path"       : rel,
                        "Page"       : pg_no,
                        "HitType"    : hit_type,
                        "MatchedText": m.group(0),
                        "StartIdx"   : s,
                        "EndIdx"     : e,
                        "Location"   : f"Page {pg_no}, line {line} (char {col})",
                        "Snippet"    : snippet
                    })

    if rows:
        append_rows_to_csv(running_csv, rows, header_order)

    # checkpoint only when not rescan-all
    if not RESCAN_ALL:
        with open(done_file, "a") as f:
            f.write(rel + "\n")

# ── CSV → Excel (always show if exists) ───────────────────────
if running_csv.exists():
    df_final = pd.read_csv(running_csv)
    display(df_final.head(50))
    df_final.to_excel(final_xlsx, index=False)
    print("\n✅ Excel saved →", final_xlsx)
else:
    print("\nℹ️  No hits written this run, and no existing CSV at:", running_csv)
    print("   • If you want to force a fresh scan, keep RESCAN_ALL=True (already set).")
    print("   • If nothing appears, confirm there are PDFs in the folder and that text/OCR is extractable.")


🔍 Scanning: /content/drive/MyDrive/UIP - OPM/OIP 25-017-Justin Wenig_3Jun25
📄 PDFs found: 16  |  Skipped (done): 0  |  Will scan now: 16
↻ RESCAN_ALL=True → ignoring __done.txt for this run.
[1/16] Microsoft 224/OPM 224 DOC - CONTRACT FINAL.pdf
[2/16] Microsoft 224/OPM 224 DOC - PO 301824, Zones LLC.pdf
[3/16] Ocelot 210/OPM 210 DOC - POA 258761.pdf
[4/16] Ocelot 210/Ocelot - POA 258761_reviewed for UIP -10-18-25.pdf
[5/16] Ocelot 210/OPM 210 DOC - Bundled.pdf


KeyboardInterrupt: 

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Checkpointed Privacy Sweep – detailed, one-row-per-hit      ║
# ╚══════════════════════════════════════════════════════════════╝
import pathlib, re, csv, pdfplumber, pandas as pd, logging, warnings, pytesseract, tempfile
from pdf2image import convert_from_path
from IPython.display import display

# ── EDIT HERE ──────────────────────────────────────────────────
ROOT_PATH      = pathlib.Path("/content/drive/MyDrive/UIP - OPM")
SUB_FOLDER_KEY = "OIP-25-003-JustinWenig_Scholarship-DonorMGT_19May25"      # part/full folder name
SNIP_LEN       = 90                            # centered snippet width
RESCAN_ALL     = True                          # <-- set True to ignore __done.txt and rescan everything
RUN_BASENAME   = "privacy_hits_detailed"       # base name for CSV/XLSX
# ───────────────────────────────────────────────────────────────

logging.getLogger("pdfminer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="pdfminer")

# ── REGEX (updated) ────────────────────────────────────────────
RX_EIN_FEIN       = re.compile(r"\b(?:FE?IN)[^\d]{0,6}?(\d{2}[- ]?\d{7})\b|\b(\d{2}[- ]?\d{7})\b", re.I)
RX_PROPRIETARY    = re.compile(r"\bProprietary\b", re.I)
RX_CONFIDENTIAL   = re.compile(r"\bConfidential\b", re.I)
RX_EMAIL          = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.I)

# PHONE: robust (+1, () area code, unicode dashes, separators, optional ext; avoids bare 10-digit runs)
_RX_SEP = r"[ \.\-_/–—]"
RX_PHONE = re.compile(
    r"(?<!\d)"
    r"(?:\+?1" + _RX_SEP + r"*)?"
    r"(?:(?:\(\s*\d{3}\s*\)" + _RX_SEP + r"*\d{3}" + _RX_SEP + r"*\d{4})"
    r"|(?:\d{3}" + _RX_SEP + r"+\d{3}" + _RX_SEP + r"+\d{4}))"
    r"(?:" + _RX_SEP + r"*(?:x|ext\.?|extension)\s*\d{1,5})?"
    r"(?!\d)", re.I
)

RX_ADDRESS        = re.compile(r"\b\d+\s+\w+\s+(?:St\.?|Street|Ave\.?|Avenue|Rd\.?|Road)\b.*\b[A-Z]{2}\s*\d{5}\b", re.I)
RX_VENDOR_REF     = re.compile(
    r"\b("
    r"vendor\s+reference|references?|reference\s+list|"
    r"customer\s+reference|client\s+reference|"
    r"past\s+performance|performance\s+history|"
    r"experience\s+record|experience\s+list|"
    r"similar\s+projects?|previous\s+(work|projects?|contracts?)|"
    r"completed\s+projects?|worked\s+with|has\s+performed|performed\s+(work|services)"
    r")\b", re.I)
RX_ATTY_CLIENT    = re.compile(r"attorney[-\s]?client", re.I)

# ── OCR helper ─────────────────────────────────────────────────
def ocr_page(pdf_path, num):
    with tempfile.TemporaryDirectory() as td:
        imgs = convert_from_path(str(pdf_path), dpi=300, fmt="png",
                                 first_page=num, last_page=num,
                                 output_folder=td)
        return pytesseract.image_to_string(imgs[0]) if imgs else ""

# ── util: line/col + snippet ──────────────────────────────────
def _line_col(text:str, idx:int):
    line = text.count("\n", 0, idx) + 1
    last_nl = text.rfind("\n", 0, idx)
    col = (idx - last_nl) if last_nl != -1 else (idx + 1)
    return line, col

def _center_snip(text:str, start:int, window:int):
    half = max(12, window // 2)
    a = max(0, start - half); b = min(len(text), start + half)
    snip = text[a:b].replace("\n"," ").strip()
    return ("…" if a > 0 else "") + snip + ("…" if b < len(text) else "")

# ── which patterns to emit as rows (order controls grouping) ──
HIT_PATTERNS = [
    ("Email",        RX_EMAIL),
    ("Phone",        RX_PHONE),
    ("EIN/FEIN",     RX_EIN_FEIN),
    ("Proprietary",  RX_PROPRIETARY),
    ("Confidential", RX_CONFIDENTIAL),
    ("Address",      RX_ADDRESS),
    ("Vendor Ref",   RX_VENDOR_REF),
    ("Atty Client",  RX_ATTY_CLIENT),
]

# ── locate target folder ──────────────────────────────────────
target = next((d for d in ROOT_PATH.iterdir()
               if d.is_dir() and SUB_FOLDER_KEY.lower() in d.name.lower()), None)
if not target:
    raise FileNotFoundError(f"No sub-folder under {ROOT_PATH} contains “{SUB_FOLDER_KEY}”")

print("🔍 Scanning:", target)

done_file   = target / "__done.txt"
running_csv = target / f"{RUN_BASENAME}.csv"
final_xlsx  = target / f"{RUN_BASENAME}_{target.name}.xlsx"

processed = set()
if done_file.exists() and not RESCAN_ALL:
    processed = set(p.strip() for p in done_file.read_text().splitlines() if p.strip())

# inventory + progress info
pdfs = list(target.rglob("*.pdf"))
skipped = sum(1 for p in pdfs if p.relative_to(target).as_posix() in processed)
to_scan = [p for p in pdfs if p.relative_to(target).as_posix() not in processed or RESCAN_ALL]

print(f"📄 PDFs found: {len(pdfs)}  |  Skipped (done): {skipped}  |  Will scan now: {len(to_scan)}")
if RESCAN_ALL:
    print("↻ RESCAN_ALL=True → ignoring __done.txt for this run.")

def append_rows_to_csv(path: pathlib.Path, rows, header):
    write_header = not path.exists()
    with open(path, "a", newline='', encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=header)
        if write_header:
            w.writeheader()
        w.writerows(rows)

# ── main scan loop (row per hit) ──────────────────────────────
header_order = ["Folder","File","Path","Page","HitType","MatchedText",
                "StartIdx","EndIdx","Location","Snippet"]

for idx, pdf_path in enumerate(to_scan, 1):
    rel = pdf_path.relative_to(target).as_posix()
    print(f"[{idx}/{len(to_scan)}] {rel}")
    rows = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for pg_no, page in enumerate(pdf.pages, 1):
            txt = page.extract_text() or ""
            if not txt.strip():
                txt = ocr_page(pdf_path, pg_no)
            if not txt:
                continue

            for hit_type, pat in HIT_PATTERNS:
                for m in pat.finditer(txt):
                    s, e = m.span()
                    line, col = _line_col(txt, s)
                    snippet = _center_snip(txt, s, SNIP_LEN)
                    rows.append({
                        "Folder"     : target.name,
                        "File"       : pdf_path.name,
                        "Path"       : rel,
                        "Page"       : pg_no,
                        "HitType"    : hit_type,
                        "MatchedText": m.group(0),
                        "StartIdx"   : s,
                        "EndIdx"     : e,
                        "Location"   : f"Page {pg_no}, line {line} (char {col})",
                        "Snippet"    : snippet
                    })

    if rows:
        append_rows_to_csv(running_csv, rows, header_order)

    # checkpoint only when not rescan-all
    if not RESCAN_ALL:
        with open(done_file, "a") as f:
            f.write(rel + "\n")

# ── CSV → Excel (always show if exists) ───────────────────────
if running_csv.exists():
    df_final = pd.read_csv(running_csv)
    display(df_final.head(50))
    df_final.to_excel(final_xlsx, index=False)
    print("\n✅ Excel saved →", final_xlsx)
else:
    print("\nℹ️  No hits written this run, and no existing CSV at:", running_csv)
    print("   • If you want to force a fresh scan, keep RESCAN_ALL=True (already set).")
    print("   • If nothing appears, confirm there are PDFs in the folder and that text/OCR is extractable.")


🔍 Scanning: /content/drive/MyDrive/UIP - OPM/OIP-25-003-JustinWenig_Scholarship-DonorMGT_19May25
📄 PDFs found: 42  |  Skipped (done): 0  |  Will scan now: 42
↻ RESCAN_ALL=True → ignoring __done.txt for this run.
[1/42] Reviewed for UIP/NextGen Approved PO 290821 (C250052)reviewed for UIP_10-18-25.pdf
[2/42] Reviewed for UIP/NextGen Executed Agreement - Contract C250052_reviewed for UIP_10-18-25.pdf
[3/42] Reviewed for UIP/C210050 - Next Gen Web Solutions_reiviewed for UIP_10-18-25.pdf
[4/42] Reviewed for UIP/ ELLUCIAN _PO_11755154_1753386615044_signed_7-24-25_Reviewed for UIP 10-18-25.pdf
[5/42] Reviewed for UIP/EllucianOrderFormODTrenewal2025August1-2026July31_REviewed for UIP_10-18-25.pdf
[6/42] OPM-4168 (SA) ELLUCIAN-ITS_EX/OPM4168 ELLUCIAN EX - ITS-PURAP_PO_11755154_1753386615044.pdf
[7/42] OPM-4168 (SA) ELLUCIAN-ITS_EX/OPM4168 ELLUCIAN EX - ITS-PURAP_PO_11755154_1753386615044_signed_7-24-25.pdf
[8/42] OPM-4168 (SA) ELLUCIAN-ITS_EX/OPM4168 Form 95_7-22-25.pdf
[9/42] OPM-4168 (SA) E

,Folder,File,Path,Page,HitType,MatchedText,StartIdx,EndIdx,Location,Snippet
0,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Approved PO 290821 (C250052)reviewed f...,Reviewed for UIP/NextGen Approved PO 290821 (C...,1,Phone,808-956-6897,1270,1282,"Page 1, line 68 (char 46)","…For more information comtact: Yamada, Janice ..."
1,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,1,Phone,(808) 932-7137,507,521,"Page 1, line 12 (char 8)",…Title: Assoc. VP for Student Affairs Phone: (...
2,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,1,Phone,(808) 956-6897,687,701,"Page 1, line 23 (char 1)",…ng Cuntact Information (if different): Ne me ...
3,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,1,Proprietary,proprietary,810,821,"Page 1, line 29 (char 95)",…d use in its own business opcrations certain ...
4,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,1,Vendor Ref,reference,1188,1197,"Page 1, line 32 (char 64)",…s attached hereto and incorporated herein by ...
5,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,2,Proprietary,proprietary,1454,1465,"Page 2, line 37 (char 39)","…w-how, mora) phy, coatract rights, and other ..."
6,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,2,Confidential,Confidential,218,230,"Page 2, line 10 (char 6)",…b Solutions pursuant to this Agresment 1.2 “...
7,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,2,Confidential,confidential,615,627,"Page 2, line 16 (char 50)",…onab'¢ person to believe such information is ...
8,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,3,Proprietary,proprietary,1369,1380,"Page 3, line 37 (char 15)","…. The Software, Documentation, Services, all ..."
9,OIP-25-003-JustinWenig_Scholarship-DonorMGT_19...,NextGen Executed Agreement - Contract C250052_...,Reviewed for UIP/NextGen Executed Agreement - ...,3,Proprietary,proprietary,2052,2063,"Page 3, line 52 (char 43)",…er grants fo SO ‘Web Solutions all necessary ...



✅ Excel saved → /content/drive/MyDrive/UIP - OPM/OIP-25-003-JustinWenig_Scholarship-DonorMGT_19May25/privacy_hits_detailed_OIP-25-003-JustinWenig_Scholarship-DonorMGT_19May25.xlsx
